In [ ]:
import sys
del sys.path[5]
del sys.path[5]
sys.path.insert(0,'/home/pgaemers/software/epix')

In [ ]:
import time
import pandas as pd
import straxen
import epix
from epix.simulator.fast_simulator import StraxSimulator

print(f'epix version : {epix.__version__}')

def monitor_time(prev_time, task):
    t = time.time()
    print(f'It took {(t - prev_time):.4f} sec to {task}')
    return t
# MC output file as an input for epix/fast_simulator

raw_data_dir='/dali/lgrandi/xenonnt/simulations/testing_epix_wfsim/'
raw_data_filename='tpc_and_nveto_cryoneutrons_200.root'
st = straxen.contexts.xenonnt_simulation(output_folder='/dali/lgrandi/pgaemers/strax_data_fast_sim')
run_id = '1'
st.register(StraxSimulator)
st.set_config=(dict(nchunk=1, event_rate=5, chunk_size=500,))

In [ ]:
# %rm -r './strax_data'

nt_config=straxen.get_resource('/home/pgaemers/software/private_nt_aux_files/\
sim_files/fax_config_nt_low_field.json', fmt='json')

# epix config

epix_args={'path':raw_data_dir,
           'file_name':raw_data_filename, 
           'debug':True,           
           'entry_start':0,
           'entry_stop':100000,
           'cut_by_eventid':False,
           'micro_separation_time':10.0,
           'micro_separation':0.005,
           'tag_cluster_by':'time',
           'max_delay':1e-7,
           'source_rate':-1}

# configuration files; to be moved into the plugin

configuration_files={'s1_relative_lce_map':straxen.pax_file('XENON1T_s1_xyz_lce_true_kr83m_SR1_pax-680_fdc-3d_v0.json'),
                     's2_xy_correction_map':straxen.pax_file('XENON1T_s2_xy_ly_SR1_v2.2.json'),
                     'photon_area_distribution':'XENONnT_spe_distributions_20210305.csv',
                     's1_pattern_map':'XENONnT_s1_xyz_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     's2_pattern_map':'XENONnT_s2_xy_patterns_LCE_corrected_qes_MCva43fa9b_wires.pkl',
                     'nv_pmt_qe':'nveto_pmt_qe.json'}

st.config.update(dict(fax_config=nt_config,
                      g4_file=raw_data_dir+raw_data_filename,
                      epix_config=epix_args,
                      configuration_files=configuration_files))

In [ ]:
st.register(StraxSimulator)

In [ ]:
st.make(run_id,'events_tpc')